In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from google.colab import files


#mounting with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = 'https://drive.google.com/drive/folders/1DlTaQRIN944WqasRoww-X8S_znufVCFT?usp=drive_link'
test_dir = 'https://drive.google.com/drive/folders/1sMu6TWzF9M8M0h84rEbl_Nr5eUg81T24?usp=sharing'

In [ ]:
def load_images(directory):
    images = []
    labels = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = image.load_img(img_path, target_size=(224, 224))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        images.append(img)

        # Infer label from filename
        if 'cat' in img_name.lower():
            labels.append(0)
        elif 'dog' in img_name.lower():
            labels.append(1)

    return np.array(images), np.array(labels)

In [ ]:
X, y = load_images(train_dir)
print(X)

In [ ]:
test_images = []
test_img_names = os.listdir(test_dir)
for img_name in test_img_names:
    img_path = os.path.join(test_dir, img_name)
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    test_images.append(img)
test_images = np.array(test_images)

In [ ]:
X_flat = X.reshape(X.shape[0], -1)
test_images_flat = test_images.reshape(test_images.shape[0], -1)

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_flat, y, test_size=0.2, random_state=42)


In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
test_images_scaled = scaler.transform(test_images_flat)

In [ ]:
# Train SVM classifier
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train_scaled, y_train)


SVC(kernel='linear', probability=True)

In [ ]:
# Evaluate on validation set
y_val_pred = svm.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Validation Accuracy: {accuracy}')

Validation Accuracy: 0.7307692307692307


In [ ]:
# Predict on test set
test_predictions = svm.predict_proba(test_images_scaled)[:, 1]
binary_test_predictions = (test_predictions >= 0.5).astype(int)

In [ ]:
#Submission
submission = pd.DataFrame({'id': [os.path.splitext(name)[0] for name in test_img_names], 'label': binary_test_predictions})

submission.to_csv('cd_.csv', index=False)
files.download('cd_.csv')
print(submission)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

     id  label
0     1      0
1     3      0
2     2      0
3    50      0
4    11      0
..   ..    ...
104  77      0
105  98      0
106  99      0
107  96      0
108  97      0

[109 rows x 2 columns]
